In [2]:
import os
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from contextlib import contextmanager
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil

%matplotlib inline


# ref: Kaggleコード遺産 https://qiita.com/kaggle_grandmaster-arai-san/items/d59b2fb7142ec7e270a5 
class Timer:
    def __init__(self, logger=None, format_str="{:.3f}[s]", prefix=None, suffix=None, sep=" "):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)


def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
# 再現性確保!
seed_everything(427)

from pathlib import Path
# data_dir
DATA_DIR = Path("../data/")

# 学習用のログデータと正解ラベル
train_log_df = pd.read_csv(DATA_DIR / "train_log.csv")
train_label_df = pd.read_csv(DATA_DIR / "train_label.csv")

# 宿のデータ
yado_df = pd.read_csv(DATA_DIR / "yado.csv")

# テスト期間のログデータ
test_log_df = pd.read_csv(DATA_DIR / "test_log.csv")

sample_submission_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

# 画像のデータ
image_df = pd.read_parquet(DATA_DIR / "image_embeddings.parquet")

# すべてのログデータはあとあと参照をするので先に作っておきます.
whole_log_df = pd.concat([train_log_df, test_log_df], ignore_index=True)

In [11]:

def calc_area(area: str, top:int) -> float:

    # 宿の出現回数を計算
    yad_count = train_log_df.groupby("yad_no").size().rename("yad_count").reset_index()

    # 宿のマスター情報を紐づけて
    _df = pd.merge(yad_count, yado_df, on="yad_no", how="left")

    # 出現回数が多い順に並び替え
    _df = _df.sort_values("yad_count", ascending=False)

    # この状態でareaCDごとに上位top件を取得する
    area_top = _df.groupby(area).head(top)

    # セッションの一番最後の宿に, yado情報を紐付ける
    session_last = pd.merge(train_log_df.groupby("session_id").last().reset_index(), 
                            yado_df, on="yad_no", how="left")

    # areaの上位topこの宿を一番最後の宿情報 `session_last` に紐づけ
    out_df = pd.merge(session_last[["session_id", area]], 
            area_top[[area, "yad_no"]], on=area, how="left")

    # 使うのはセッションと宿の関係
    out_df = out_df[["session_id", "yad_no"]].copy()
    
    # ランダムに付け加えたもの以外・同一ログに出現する宿を候補にいれる
    out_df = pd.concat([out_df, train_log_df[["session_id", "yad_no"]]], ignore_index=True)
    
    # 重複は意味がないので消します。
    out_df = out_df.drop_duplicates()
    
    # 見た目を揃えるために session / yad の順番でソートをします
    out_df = out_df.sort_values(["session_id", "yad_no"]).reset_index(drop=True)

    _df = pd.merge(out_df[["session_id"]], 
               train_label_df, on="session_id", how="left")
    has_truth_yado = (out_df["yad_no"] == _df["yad_no"]).groupby(out_df["session_id"]).sum()

    return has_truth_yado.mean()

In [14]:
calc_area("ken_cd", 100)

0.7326306382448095

In [21]:
calc_area("ken_cd", 200)

0.8606190552064787

In [15]:
calc_area("lrg_cd", 30)

0.7402025646176974

In [17]:
calc_area("lrg_cd", 50)

0.837020000138553

In [18]:
calc_area("lrg_cd", 100)

0.9195699312083908

In [16]:
calc_area("sml_cd", 30)

0.7311308010446903

In [19]:
calc_area("sml_cd", 50)

0.7948063374183403

In [20]:
calc_area("sml_cd", 100)

0.8344533041448157